# Laboratorio de regresión - 5

|                |   |
:----------------|---|
| **Nombre**     |   |
| **Fecha**      |   |
| **Expediente** |   |

## Validación

Hemos estado usando `train_test_split` en nuestros modelos anteriores.

¿Por qué?

Si la muestra es un subset de la población y queremos generalizar sobre la población, ¿no sería mejor utilizar todos los datos al entrenar un modelo?

El propósito de volver a muestrear dentro de nuestro dataset es tener una idea de qué tan buena podría ser la generalización de nuestro modelo. Imagina un dataset ya separado en dos mitades. Utilizas la primera mitad para entrenar el modelo y pruebas en la segunda mitad; la segunda mitad eran datos invisibles para el modelo al momento de entrenar. Esto nos lleva a tres escenario típicos:

1. Si el modelo hace buenas predicciones en la segunda mitad, significa que la primera mitad era "suficiente" para generalizar.
2. Si el modelo no hace buenas predicciones en la segunda mitad, pero sí en la primera mitad, podría ser que había información importante en la segunda mitad que debió haber sido tomada en cuenta al entrenar, o un problema de overfitting.
3. Si el modelo no hace buenas predicciones en la segunda mitad, y tampoco en la primera mitad, se tendrían que revisar los factores y/o el modelo seleccionado.

El caso ideal sería el 1, pero por estadística los errores y varianzas tienen como entrada el número de muestas, por lo que tenemos menos seguridad de nuestros resutados al usar menos muestras. Si vemos que el modelo generaliza bien podemos unir de nuevo el dataset y entrenar sobre el dataset completo.

En el caso 2 está el problema de que no podemos saber qué información es necesaria para el entrenamiento apropiado del modelo; esto nos lleva a pensar que debemos usar el dataset completo para entrenar, pero esto nos lleva al mismo problema de no saber si el modelo puede generalizar.

El problema sólo incrementa si se tienen hiperparámetros en el modelo (e.g. $\lambda$ en regularización).

## Leave-One-Out Cross Validation

Este método de validación es una colección de $n$ `train-test-split`. Teniendo un dataset de $n$ muestras, la lógica es:
1. Saca una muestra del dataset.
2. Entrena tu modelo con las $n-1$ muestras.
3. Evalúa tu modelo en la muestra que quedó fuera con el métrico que más se ajuste a la aplicación.
4. Regresa la muestra al dataset.
5. Repite 1-4 con muestras diferentes hasta haber hecho el procedimiento $n$ veces para $n$ muestras.
6. Calcula la media y desviación estándar de los métricos guardados.

Con los resultados del proceso de validación podemos saber qué tan bueno podría ser el modelo seleccionado con los datos (con/sin transformaciones).

### Ejercicio 1

Utiliza el dataset `Motor Trend Car Road Tests`. Elimina la columna `model` y entrena 32 modelos diferentes utilizando Leave-One-Out Cross Validation con target `mpg`. Utiliza MSE como métrico.

In [118]:
import pandas as pd
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [120]:
df=pd.read_excel("C:/Users/munoz/Downloads/lab_apre_est/Motor Trend Car Road Tests.xlsx")
df.head()

,model,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


In [122]:
df2=df.drop(columns=["model"])
df2.head()

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


In [132]:
mses = []

for i in range(32):

    datos_train = df2.drop(index=i)
    datos_test = df2.iloc[[i]]
    
    X_train = datos_train.drop(columns=["mpg"])
    y_train = datos_train["mpg"].values
    X_test = datos_test.drop(columns=["mpg"])
    y_test = datos_test["mpg"].values

    scaler = StandardScaler().fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_train_sm = sm.add_constant(X_train_scaled)
    
    modelo_mpg = sm.OLS(y_train, X_train_sm).fit()
 
    X_test_scaled = scaler.transform(X_test)
    X_test_sm = sm.add_constant(X_test_scaled, has_constant='add')
    y_pred = modelo_mpg.predict(X_test_sm)
    
    MSE = mean_squared_error(y_test, y_pred)
    MSE_lista.append(MSE)

MSE_lista
print("El MSE promedio es: ", np.mean(MSE_lista))

#La desviación estándar del MSE es:
print("La desviación estándar del MSE es: ", np.std(MSE_lista))


El MSE promedio es:  12.181558006901938
La desviación estándar del MSE es:  17.067399871888526


In [124]:

mses=[]

In [126]:

for i in range(32):
    train_df2=df2.drop(i)
    test_df2=df2.iloc[[i]]

    colum=df2.columns
    colum=colum.drop("mpg")
    xtrain_list=[]
    xtest_list=[]
 
    
    for col in df2:
        xtrain_list.append(train_df2[col].values.reshape(-1,1))
        xtest_list.append(test_df2[col].values.reshape(-1,1))
        
    xtrain= np.hstack(xtrain_list)
    xtest=np.hstack(xtest_list)
    ytrain=train_df2["mpg"].values.reshape(-1,1)
    ytest=test_df2["mpg"].values.reshape(-1,1)

    scaler=StandardScaler().fit(xtrain)
    xscaled_train=scaler.transform(xtrain)

    X=np.ones((len(xtrain),1))
    xetrain=np.hstack((X,xscaled_train))

    ols=sm.OLS(ytrain,xetrain)
    resultados=ols.fit()

    xscaled_test=scaler.transform(xtest)
    x1=np.ones((len(xtest),1))
    xetest=np.hstack((x1,xscaled_test))

    y_pred=resultados.predict(xetest)
    mse=mean_squared_error(ytest,y_pred)
    mses.append(mse)

print("MES promedio", np.mean(mses))
print("Desviación estándar de MSE: ",np.std(mses))




MES promedio 1.5934990285464438e-28
Desviación estándar de MSE:  1.5013662424428168e-28


In [128]:
print(mses)
print(np.mean(mses))
print(np.std(mses))

[2.0194839173657902e-28, 1.135959703518257e-28, 2.0194839173657902e-28, 5.048709793414476e-29, 2.0194839173657902e-28, 4.543838814073028e-28, 1.135959703518257e-28, 0.0, 1.262177448353619e-29, 5.048709793414476e-29, 5.048709793414476e-29, 1.135959703518257e-28, 4.543838814073028e-28, 1.135959703518257e-28, 1.5461673742331831e-28, 2.0194839173657902e-28, 2.0194839173657902e-28, 0.0, 2.0194839173657902e-28, 5.048709793414476e-29, 0.0, 2.5559093329160782e-28, 3.1554436208840472e-30, 3.818086781269697e-28, 2.0194839173657902e-28, 1.135959703518257e-28, 2.0194839173657902e-28, 5.048709793414476e-29, 6.184669496932733e-28, 0.0, 1.262177448353619e-29, 3.155443620884047e-28]
1.5934990285464438e-28
1.5013662424428168e-28


Interpreta.

## K-Folds Cross-Validation

El dataset `Motor Trend Car Road Tests` sólo tiene 32 muestras, y utilizar un modelo sencillo de regresión múltiple hace que usar LOOCV sea muy rápido. El dataset `California Housing` tiene $20640$ muestras para $9$ columnas, entonces realizar un ajuste sobre una transformación o sobre el modelo y luego calcular el impacto esperado podría tomar más tiempo.

La solución propuesta es dividir el dataset en *k* folds (partes iguales), ajustar en *k-1* folds y probar en el restante.

### Ejercicio 2
Utiliza el dataset `California Housing` y haz K-folds Cross Validation con 10 folds. Utiliza el MSE como métrico.

In [ ]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
print("Dataset Shape:", housing.data.shape, housing.target.shape)
print("Dataset Features:", housing.feature_names)
print("Dataset Target:", housing.target_names)
X = housing.data
y = housing.target

Interpreta.

## Referencia

James, G., Witten, D., Hastie, T., Tibshirani, R.,, Taylor, J. (2023). An Introduction to Statistical Learning with Applications in Python. Cham: Springer. ISBN: 978-3-031-38746-3